<a href="https://colab.research.google.com/github/wilberquito/melanoma.thesis/blob/main/Code/pytorch/notebooks/Models_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models Analysis

In [1]:
!pip install wandb torchinfo >/dev/null 2>&1

In [2]:
try:
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend verison should be 0.19.0 or higher"
except:
    !pip install -q torchmetrics -U mlxtend # <- Note: If you're using Google Colab, this may require restarting the runtime
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 8.8 MB/s eta 0:00:00
mlxtend version: 0.22.0


In [4]:
import requests
from pathlib import Path
import pandas as pd
import seaborn as sns
from zipfile import ZipFile

try:
    import google.colab
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
except:
    IN_COLAB = False

print(f"In colab: {IN_COLAB}")

Mounted at /content/drive
In colab: True


In [5]:
def pull_code():
  print("Unzipping the code from google drive")
  !rm -rf modular/
  !unzip ./drive/MyDrive/wilberquito_thesis/modular.zip -d . >/dev/null 2>&1

if IN_COLAB:
    pull_code()

Unzipping the code from google drive


## Transfer Learning Model (ResNet-18)

In [6]:
import torchinfo
from torchvision.models import (resnet18,
                                ResNet18_Weights)

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

torchinfo.summary(model,
                  (3, 256, 256),
                  batch_dim = 0,
                  depth=2,
                  col_names = ("input_size", 'output_size', 'num_params'),
                  verbose = 0)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 111MB/s]


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
ResNet                                   [1, 3, 256, 256]          [1, 1000]                 --
├─Conv2d: 1-1                            [1, 3, 256, 256]          [1, 64, 128, 128]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 128, 128]         [1, 64, 128, 128]         128
├─ReLU: 1-3                              [1, 64, 128, 128]         [1, 64, 128, 128]         --
├─MaxPool2d: 1-4                         [1, 64, 128, 128]         [1, 64, 64, 64]           --
├─Sequential: 1-5                        [1, 64, 64, 64]           [1, 64, 64, 64]           --
│    └─BasicBlock: 2-1                   [1, 64, 64, 64]           [1, 64, 64, 64]           73,984
│    └─BasicBlock: 2-2                   [1, 64, 64, 64]           [1, 64, 64, 64]           73,984
├─Sequential: 1-6                        [1, 64, 64, 64]           [1, 128, 32, 32]          --
│    └─BasicBlock: 2-3 

## ResNet18_Melanoma

In [7]:
import torchinfo
import modular.models as m_models

out_size = 8
model = m_models.ResNet18_Melanoma(out_size)

torchinfo.summary(model,
                  (3, 256, 256),
                  batch_dim = 0,
                  depth=2,
                  col_names = ("input_size", 'output_size', 'num_params'),
                  verbose = 0)

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #
ResNet18_Melanoma                             [1, 3, 256, 256]          [1, 8]                    --
├─ResNet: 1-1                                 [1, 3, 256, 256]          [1, 512]                  --
│    └─Conv2d: 2-1                            [1, 3, 256, 256]          [1, 64, 128, 128]         9,408
│    └─BatchNorm2d: 2-2                       [1, 64, 128, 128]         [1, 64, 128, 128]         128
│    └─ReLU: 2-3                              [1, 64, 128, 128]         [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                         [1, 64, 128, 128]         [1, 64, 64, 64]           --
│    └─Sequential: 2-5                        [1, 64, 64, 64]           [1, 64, 64, 64]           147,968
│    └─Sequential: 2-6                        [1, 64, 64, 64]           [1, 128, 32, 32]          525,568
│    └─Sequential: 2-7                        [1, 128, 32, 32]          

In [8]:
import torchinfo
import modular.models as m_models

out_size = 8
model = m_models.ResNet18_Dropout_Melanoma(out_size)

torchinfo.summary(model,
                  (3, 256, 256),
                  batch_dim = 0,
                  depth=2,
                  col_names = ("input_size", 'output_size', 'num_params'),
                  verbose = 0)

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #
ResNet18_Dropout_Melanoma                     [1, 3, 256, 256]          [1, 8]                    --
├─ResNet: 1-1                                 [1, 3, 256, 256]          [1, 512]                  --
│    └─Conv2d: 2-1                            [1, 3, 256, 256]          [1, 64, 128, 128]         9,408
│    └─BatchNorm2d: 2-2                       [1, 64, 128, 128]         [1, 64, 128, 128]         128
│    └─ReLU: 2-3                              [1, 64, 128, 128]         [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                         [1, 64, 128, 128]         [1, 64, 64, 64]           --
│    └─Sequential: 2-5                        [1, 64, 64, 64]           [1, 64, 64, 64]           147,968
│    └─Sequential: 2-6                        [1, 64, 64, 64]           [1, 128, 32, 32]          525,568
│    └─Sequential: 2-7                        [1, 128, 32, 32]          